# Multi-layer merge by centroid script

This script runs a single layer join on hex grid. Uses Kate's generated hexes and Inputs.<br>
There is no cleaning and type controll. No optimization/vectorization/parallelization. 

In [1]:
# Import modules
import geopandas as gpd
import fiona

Modify below cell to indicate path to database, hex level and joined layer

In [16]:
# Define path to hex grid database
hex_gdb = r'C://Research/Grid_effort/H3_5_10_Grd.gdb'
# Define hex level
hexLevel = '9'

# Define the path to the geodatabase with joined layer
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
# Names of layers to join in the list, determines the order of joins
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']

Run below cell to perform merge and save result

In [ ]:
# Read in the hex polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length', 'Shape_Area'], inplace=True)

# Calculate the centroids of the hexes for the spatial join, but keep the original geometries
base_centroids = base_layer.copy()
base_centroids['geometry'] = base_layer['geometry'].centroid

# Ensure the geometry column is set correctly for centroids
base_centroids = base_centroids.set_geometry('geometry')

# Read the joined layers into a dictionary
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

# Check if garbage data is present and try to remove it
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_centroids.crs:
        gdf = gdf.to_crs(base_centroids.crs)
    
    # Check if garbage data is present and remove it
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Check for duplicate column names and rename them before the join, but don't rename 'geometry'
    for col in gdf.columns:
        if col in base_centroids.columns and col != 'geometry':
            gdf.rename(columns={col: col + '_joined'}, inplace=True)
    
    # Set the geometry column to ensure spatial join works
    gdf = gdf.set_geometry('geometry')

    # Perform spatial join using centroids of base_layer hexes
    join_result = gpd.sjoin(base_centroids, gdf, how="left", predicate="within")

    # Transfer the join result back to the original base_layer (with full hex geometries)
    base_layer = base_layer.join(join_result.drop(columns='geometry'), rsuffix='_joined')

    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

# After the join, check for any duplicated columns and rename them to avoid conflicts
base_layer = base_layer.loc[:, ~base_layer.columns.duplicated()]

# Save the result in work path
base_layer.to_file('h'+hexLevel+'_centroid.gdb', driver='OpenFileGDB')
